## Books Database Set Up

In [28]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [23]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [24]:
!mongoimport --type csv -d books_db -c books --headerline --drop Resources/Books.csv
!mongoimport --type csv -d books_db -c users --headerline --drop Resources/Users.csv
!mongoimport --type csv -d books_db -c ratings --headerline --drop Resources/Ratings.csv

2023-12-04T21:52:15.864-0500	connected to: mongodb://localhost/
2023-12-04T21:52:15.865-0500	dropping: books_db.books
2023-12-04T21:52:15.899-0500	2 document(s) imported successfully. 0 document(s) failed to import.
2023-12-04T21:52:16.041-0500	connected to: mongodb://localhost/
2023-12-04T21:52:16.041-0500	dropping: books_db.users
2023-12-04T21:52:16.074-0500	2 document(s) imported successfully. 0 document(s) failed to import.
2023-12-04T21:52:16.223-0500	connected to: mongodb://localhost/
2023-12-04T21:52:16.224-0500	dropping: books_db.ratings
2023-12-04T21:52:16.262-0500	2 document(s) imported successfully. 0 document(s) failed to import.


In [25]:
# confirm that our new database was created
mongo.list_database_names()

['admin', 'books_db', 'config', 'local', 'met', 'uk_food']

In [26]:
# assign the uk_food database to a variable name
db = mongo['books_db']

In [27]:
# review the collections in our new database
db.list_collection_names()

['ratings', 'books', 'users']

In [31]:
# assign the collection to a variable
books=db['books']
ratings=db['ratings']
users=db['users']

In [35]:
print("Example of a record within 'books' collection: ")
pprint(books.find_one({}))
print("\n Example of a record within 'ratings' collection:")
pprint(ratings.find_one({}))
print("\n Example of a record within 'users' collection:")
pprint(users.find_one({}))

Example of a record within 'books' collection: 
{'Book-Author': 'Mark P. O. Morford',
 'Book-Title': 'Classical Mythology',
 'ISBN': 195153448,
 'Publisher': 'Oxford University',
 'Year-Of-Publication': 2002,
 '_id': ObjectId('656e905f47e960c192f944c0')}

 Example of a record within 'ratings' collection:
{'Book-Rating': 2,
 'ISBN': 195153448,
 'User-ID': 1,
 '_id': ObjectId('656e9060d0c44eebb0c523d9')}

 Example of a record within 'users' collection:
{'Age': 34,
 'Location': 'Mexico',
 'User-ID': 2,
 '_id': ObjectId('656e9060c05552a062fb3c84')}
